In [1]:
# !ls adversarial-robustness-toolbox/
import os
os.chdir('/home/surthi/adversarial-robustness-toolbox/')

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
import tensorflow as tf
import keras.backend as k
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from keras.regularizers import l2
import numpy as np
from art.attacks import DeepFool
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from art.data_generators import KerasDataGenerator
from art.defences import AdversarialTrainer


Using TensorFlow backend.


In [3]:
#https://keras.io/examples/cifar10_resnet/
def evaluate(clf, x_train, y_train, x_test, y_test):
    # train acc
    train_preds = np.argmax(clf.predict(x_train), axis=1)
    train_acc = np.sum(train_preds == np.argmax(y_train, axis=1)) / y_train.shape[0]
    # test acc
    test_preds = np.argmax(clf.predict(x_test), axis=1)
    test_acc = np.sum(test_preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
    print("\nTraining and Test accuracies: %.2f %.2f%%" % (train_acc*100, test_acc*100))
    return train_acc, test_acc

def plot_curves(model, title='model accuracy and loss'):
    import matplotlib.pyplot as plt
    plt.plot(model.history.history['acc'])
    plt.plot(model.history.history['loss'])
    plt.title(title)
    plt.xlabel('Epoch')
    plt.legend(['Accuracy', 'Loss'], loc='upper left')
    return plt

def save_clf(c, folder, clf_filename, model_filename):
    cwd = os.getcwd()
    os.chdir(folder)
    import pickle
    with open(clf_filename, "wb") as f:
        pickle.dump(c.__getstate__, f)
    c._model.save(model_filename)
    os.chdir(cwd)

DUMP_FOLDER = '/home/surthi/models/'
def pickle_dump(data, filename, folder=DUMP_FOLDER):
    cwd = os.getcwd()
    os.chdir(folder)
    import pickle
    with open(filename, "wb") as f:
        pickle.dump(data, f)
    os.chdir(cwd)

def pickle_load(filename):
    import pickle
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

def load_clf(folder, clf_filename, model_filename):
    cwd = os.getcwd()
    os.chdir(folder)
    import pickle
    with open(clf_filename, "rb") as f:
        clf_state = pickle.load(f)
    model = tf.keras.models.load_model(model_filename)
    os.chdir(cwd)
    clf = KerasClassifier(model=model)
    clf.__setstate__(clf_state())
    return clf, model

In [4]:
# x_train, y_train,\
# x_test, y_test,\
# x_train_fgsm_adv, x_test_fgsm_adv, \
# x_train_bim_adv, x_test_bim_adv,\
# x_train_deepfool_adv, x_test_deepfool_adv, \
# x_train_deepfool_adv_5to10, x_test_deepfool_adv_5to10, \
# x_train_cl2_adv, x_test_cl2_adv, \
# x_train_cl2_adv_5to10,x_test_cl2_adv_5to10, \
#  = pickle_load('/home/surthi/models/cifar10/vanilla_clf_adv_data.pkl')

# pickle_dump((x_train, y_train, x_test, y_test), 'vanilla_clf_train_test_data.pkl')
# pickle_dump((x_train_bim_adv, x_test_bim_adv), 'vanilla_clf_bim_xtrain_xtest.pkl')

# Load Data and Models

In [4]:
(g_x_train, g_y_train, g_x_test, g_y_test) = pickle_load('/home/surthi/models/vanilla_clf_train_test_data.pkl')
(g_x_train_bim_adv, g_x_test_bim_adv) = pickle_load('/home/surthi/models/vanilla_clf_bim_xtrain_xtest.pkl')

In [6]:
cnn_clf_bn, cnn_model_bn = load_clf('/home/surthi/models/cifar10/', 'vanilla_clf_with_bn.h5', 'vanilla_clf_with_bn_model.h5')
evaluate(cnn_clf_bn, g_x_train, g_y_train, g_x_test, g_y_test)
evaluate(cnn_clf_bn, g_x_train_bim_adv, g_y_train, g_x_test_bim_adv, g_y_test)

cnn_clf, cnn_model = load_clf('/home/surthi/models/cifar10/', 'vanilla_clf.h5', 'vanilla_clf_model.h5')
evaluate(cnn_clf, g_x_train, g_y_train, g_x_test, g_y_test)
evaluate(cnn_clf, g_x_train_bim_adv, g_y_train, g_x_test_bim_adv, g_y_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


Training and Test accuracies: 98.15 82.28%

Training and Test accuracies: 2.99 5.60%

Training and Test accuracies: 97.93 81.64%

Training and Test accuracies: 0.42 4.39%


(0.00416, 0.0439)

# Adversarial Training with 1 epoch of clean data and 1 epoch of BIM adversarial data

### Runs 10 epochs of adversarial training with 5 epochs on clean data and 5 epochs on bim-adv-data

In [8]:
def adv_training_1(clf, x_train, y_train, x_train_adv, x_test_adv, x_test, y_test, epochs=5, batch_size=128):
    print("Before training:")
    evaluate(clf, x_train, y_train, x_test, y_test)    
    evaluate(clf, x_train_adv, y_train, x_test_adv, y_test)
    
    for i in range(epochs):
        clf.fit(x_train_adv, y_train, nb_epochs=1, batch_size=128)
        clf.fit(x_train, y_train, nb_epochs=1, batch_size=128)
    
    print("After training:")
    evaluate(clf, x_train, y_train, x_test, y_test)    
    evaluate(clf, x_train_adv, y_train, x_test_adv, y_test)
    return clf

In [17]:
resnet_clf, resnet_model = load_clf('/home/surthi/models/cifar10/', 'resnet_clf.h5', 'resnet_clf_model.h5')
resnet_clf_bn, resnet_model_bn = load_clf('/home/surthi/models/cifar10/', 'resnet_clf_bn.h5', 'resnet_clf_model_bn.h5')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [29]:
import numpy.linalg as la
def loss_sensitivity(classifier, x, y):
    grads = classifier.loss_gradient(x, y)
    norm = la.norm(grads.reshape(grads.shape[0], -1), ord=2, axis=1)
    return np.mean(norm)

resnet_ls_clean_bn = loss_sensitivity(resnet_clf_bn_adv_trained, g_x_train[:3000], g_y_train[:3000])
resnet_ls_clean = loss_sensitivity(resnet_clf_adv_trained, g_x_train[:3000], g_y_train[:3000])
resnet_ls_bim_bn = loss_sensitivity(resnet_clf_bn_adv_trained, g_x_train_bim_adv[:1000], g_y_train[:1000])
resnet_ls_bim = loss_sensitivity(resnet_clf_adv_trained, g_x_train_bim_adv[:1000], g_y_train[:1000])
print(resnet_ls_clean_bn, resnet_ls_clean, resnet_ls_bim_bn, resnet_ls_bim)

5.946496 4.228237 19.853825 59.269493


In [31]:
ls_clean_bn = loss_sensitivity(cnn_clf_bn_adv_trained, g_x_train[:3000], g_y_train[:3000])
ls_clean = loss_sensitivity(cnn_clf_adv_trained, g_x_train[:3000], g_y_train[:3000])
ls_bim_bn = loss_sensitivity(cnn_clf_bn_adv_trained, g_x_train_bim_adv[:1000], g_y_train[:1000])
ls_bim = loss_sensitivity(cnn_clf_adv_trained, g_x_train_bim_adv[:1000], g_y_train[:1000])
print(ls_clean_bn, ls_clean, ls_bim_bn, ls_bim)

3.6842744 0.0 14.351504 0.0


# Adversarial training of CIFAR-RESNET and CIFAR-RESNET-With_BatchNorm

In [50]:
resnet_clf_bn_adv_trained_1 = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.31 79.14%

Training and Test accuracies: 10.40 10.79%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 1.4393 - acc: 0.6107
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4771 - acc: 0.9054
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 1.0545 - acc: 0.7174
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4167 - acc: 0.9281
Epoch 1/1
391/391 [==============================] - 40s 101ms/step - loss: 0.9100 - acc: 0.7683
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4118 - acc: 0.9296
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.8244 - acc: 0.7989
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3976 - acc: 0.9358
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.7495 - acc: 0.8220
Epoch 1/1
391/391 [===

In [51]:
resnet_clf_bn_adv_trained_1 = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.17 79.08%

Training and Test accuracies: 8.03 8.58%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.7004 - acc: 0.8422
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3941 - acc: 0.9407
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.6729 - acc: 0.8502
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3946 - acc: 0.9409
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.6350 - acc: 0.8625
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3911 - acc: 0.9430
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.6063 - acc: 0.8748
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.3929 - acc: 0.9440
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5972 - acc: 0.8778
Epoch 1/1
391/391 [=====

In [52]:
# resnet_clf_bn_adv_trained_1 = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.10 77.80%

Training and Test accuracies: 36.91 33.45%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.8079 - acc: 0.8059
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4313 - acc: 0.9373
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.7745 - acc: 0.8211
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4304 - acc: 0.9378
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.7501 - acc: 0.8295
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4293 - acc: 0.9369
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.7335 - acc: 0.8347
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4265 - acc: 0.9396
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.7279 - acc: 0.8368
Epoch 1/1
391/391 [============

In [53]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.80 77.59%

Training and Test accuracies: 38.43 34.33%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.7174 - acc: 0.8430
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4207 - acc: 0.9439
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6957 - acc: 0.8496
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4355 - acc: 0.9391
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.7014 - acc: 0.8487
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4238 - acc: 0.9429
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6731 - acc: 0.8581
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4285 - acc: 0.9420
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6805 - acc: 0.8576
Epoch 1/1
391/391 [============

In [54]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.64 77.71%

Training and Test accuracies: 45.61 39.71%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6539 - acc: 0.8656
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4296 - acc: 0.9416
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6446 - acc: 0.8707
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4253 - acc: 0.9432
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6578 - acc: 0.8642
Epoch 1/1
391/391 [==============================] - 28s 70ms/step - loss: 0.4254 - acc: 0.9446
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6390 - acc: 0.8729
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4170 - acc: 0.9479
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6557 - acc: 0.8675
Epoch 1/1
391/391 [============

In [55]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.72 77.76%

Training and Test accuracies: 47.17 40.07%
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.6349 - acc: 0.8760
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4237 - acc: 0.9459
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.6353 - acc: 0.8746
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4266 - acc: 0.9465
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.6276 - acc: 0.8790
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4259 - acc: 0.9476
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.6128 - acc: 0.8836
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4272 - acc: 0.9473
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6368 - acc: 0.8766
Epoch 1/1
391/391 [============

In [56]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 94.92 76.27%

Training and Test accuracies: 48.04 40.27%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6147 - acc: 0.8828
Epoch 1/1
391/391 [==============================] - 28s 70ms/step - loss: 0.4268 - acc: 0.9465
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6111 - acc: 0.8841
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4198 - acc: 0.9491
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6034 - acc: 0.8873
Epoch 1/1
391/391 [==============================] - 28s 70ms/step - loss: 0.4324 - acc: 0.9462
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5902 - acc: 0.8924
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4210 - acc: 0.9502
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.6094 - acc: 0.8845
Epoch 1/1
391/391 [============

In [57]:
save_clf(resnet_clf_bn, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_35_epochs.h5', 'resnet_model_adv1_trained_35_epochs.h5')
save_clf(resnet_clf, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_35_epochs.h5', 'resnet_model_adv1_trained_35_epochs.h5')
# resnet_clf._model.history

In [69]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.57 77.34%

Training and Test accuracies: 49.60 41.26%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5831 - acc: 0.8950
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4241 - acc: 0.9475
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5909 - acc: 0.8923
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4208 - acc: 0.9505
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5948 - acc: 0.8912
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4243 - acc: 0.9489
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5852 - acc: 0.8957
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4241 - acc: 0.9495
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5858 - acc: 0.8953
Epoch 1/1
391/391 [============

In [70]:
save_clf(resnet_clf_bn, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_40_epochs.h5', 'resnet_model_adv1_trained_40_epochs.h5')
save_clf(resnet_clf, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_40_epochs.h5', 'resnet_model_adv1_trained_40_epochs.h5')

In [71]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.60 76.44%

Training and Test accuracies: 51.98 42.48%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5748 - acc: 0.8984
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4238 - acc: 0.9500
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5902 - acc: 0.8943
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4278 - acc: 0.9489
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5808 - acc: 0.8972
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4196 - acc: 0.9515
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5798 - acc: 0.8971
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4177 - acc: 0.9520
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5687 - acc: 0.9012
Epoch 1/1
391/391 [============

In [72]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 97.03 77.66%

Training and Test accuracies: 52.53 42.91%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5518 - acc: 0.9077
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4161 - acc: 0.9538
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5530 - acc: 0.9061
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4181 - acc: 0.9529
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5605 - acc: 0.9052
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4213 - acc: 0.9518
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5429 - acc: 0.9105
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4112 - acc: 0.9555
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5398 - acc: 0.9110
Epoch 1/1
391/391 [============

In [73]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 97.19 77.50%

Training and Test accuracies: 58.98 45.51%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5311 - acc: 0.9145
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4064 - acc: 0.9570
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5374 - acc: 0.9109
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4044 - acc: 0.9575
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5210 - acc: 0.9180
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4115 - acc: 0.9544
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5265 - acc: 0.9164
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4118 - acc: 0.9553
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.5286 - acc: 0.9154
Epoch 1/1
391/391 [============

In [74]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.60 76.86%

Training and Test accuracies: 62.64 46.97%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5168 - acc: 0.9197
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4101 - acc: 0.9558
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5112 - acc: 0.9213
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4053 - acc: 0.9570
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5113 - acc: 0.9221
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4050 - acc: 0.9579
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5207 - acc: 0.9177
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3996 - acc: 0.9595
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5072 - acc: 0.9231
Epoch 1/1
391/391 [============

In [75]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.82 76.86%

Training and Test accuracies: 61.88 46.47%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5041 - acc: 0.9229
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4024 - acc: 0.9592
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5113 - acc: 0.9223
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3942 - acc: 0.9612
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5025 - acc: 0.9236
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4045 - acc: 0.9565
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5047 - acc: 0.9235
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.4015 - acc: 0.9588
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.5020 - acc: 0.9239
Epoch 1/1
391/391 [============

In [76]:
save_clf(resnet_clf_bn, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_100_epochs.h5', 'resnet_model_adv1_trained_100_epochs.h5')
save_clf(resnet_clf, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_100_epochs.h5', 'resnet_model_adv1_trained_100_epochs.h5')

In [77]:
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1 = adv_training_1(resnet_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.63 76.47%

Training and Test accuracies: 68.91 50.08%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4846 - acc: 0.9296
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3947 - acc: 0.9609
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4958 - acc: 0.9248
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3945 - acc: 0.9595
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4954 - acc: 0.9249
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3911 - acc: 0.9615
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4877 - acc: 0.9285
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3929 - acc: 0.9602
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.4905 - acc: 0.9271
Epoch 1/1
391/391 [============


Training and Test accuracies: 96.98 77.22%

Training and Test accuracies: 68.76 50.08%
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4777 - acc: 0.9306
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3904 - acc: 0.9605
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4671 - acc: 0.9345
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3890 - acc: 0.9620
Epoch 1/1
391/391 [==============================] - 27s 69ms/step - loss: 0.4775 - acc: 0.9315
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3878 - acc: 0.9624
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4869 - acc: 0.9261
Epoch 1/1
391/391 [==============================] - 28s 71ms/step - loss: 0.3974 - acc: 0.9584
Epoch 1/1
391/391 [==============================] - 27s 70ms/step - loss: 0.4710 - acc: 0.9330
Epoch 1/1
391/391 [=============================

In [78]:
save_clf(resnet_clf_bn, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_180_epochs.h5', 'resnet_model_adv1_trained_180_epochs.h5')
save_clf(resnet_clf, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_180_epochs.h5', 'resnet_model_adv1_trained_180_epochs.h5')

In [79]:
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 96.24 79.35%

Training and Test accuracies: 8.68 8.89%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5735 - acc: 0.8845
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3807 - acc: 0.9501
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5587 - acc: 0.8915
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3842 - acc: 0.9494
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5506 - acc: 0.8929
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3818 - acc: 0.9517
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5429 - acc: 0.8969
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3753 - acc: 0.9525
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.5268 - acc: 0.9041
Epoch 1/1
391/391 [=====


Training and Test accuracies: 11.50 11.59%
Before training:

Training and Test accuracies: 95.19 78.37%

Training and Test accuracies: 11.50 11.59%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4485 - acc: 0.9344
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3620 - acc: 0.9628
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4606 - acc: 0.9281
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3616 - acc: 0.9630
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4477 - acc: 0.9343
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.3699 - acc: 0.9603
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4549 - acc: 0.9320
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3516 - acc: 0.9669
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss:

In [ ]:
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
resnet_clf_adv_trained_1_bn = adv_training_1(resnet_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 94.67 77.68%

Training and Test accuracies: 15.94 16.17%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4509 - acc: 0.9317
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3579 - acc: 0.9647
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4569 - acc: 0.9307
Epoch 1/1
391/391 [==============================] - 40s 104ms/step - loss: 0.3567 - acc: 0.9647
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4472 - acc: 0.9344
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3600 - acc: 0.9633
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4509 - acc: 0.9329
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3647 - acc: 0.9621
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4482 - acc: 0.9337
Epoch 1/1
391/391 [===


Training and Test accuracies: 14.03 14.18%
Before training:

Training and Test accuracies: 96.56 78.83%

Training and Test accuracies: 14.03 14.18%
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4337 - acc: 0.9384
Epoch 1/1
391/391 [==============================] - 41s 104ms/step - loss: 0.3474 - acc: 0.9673
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4317 - acc: 0.9381
Epoch 1/1
391/391 [==============================] - 41s 105ms/step - loss: 0.3503 - acc: 0.9660
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4260 - acc: 0.9411
Epoch 1/1
391/391 [==============================] - 41s 105ms/step - loss: 0.3533 - acc: 0.9644
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.4314 - acc: 0.9387
Epoch 1/1
391/391 [==============================] - 41s 104ms/step - loss: 0.3505 - acc: 0.9668
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss:

After training:

Training and Test accuracies: 96.05 78.76%

Training and Test accuracies: 17.06 16.58%
Before training:

Training and Test accuracies: 96.05 78.76%

Training and Test accuracies: 17.06 16.58%
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4234 - acc: 0.9404
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3479 - acc: 0.9665
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4202 - acc: 0.9428
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3489 - acc: 0.9655
Epoch 1/1
391/391 [==============================] - 40s 101ms/step - loss: 0.4218 - acc: 0.9413
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3418 - acc: 0.9687
Epoch 1/1
391/391 [==============================] - 40s 102ms/step - loss: 0.4199 - acc: 0.9431
Epoch 1/1
391/391 [==============================] - 40s 103ms/step - loss: 0.3443 - acc: 0.9686
Epoch 1/1
391/3

In [ ]:
save_clf(resnet_clf_bn, '/home/surthi/models/cifar10/', 'resnet_clf_adv1_trained_8_epochs.h5', 'resnet_model_adv1_trained_8_epochs.h5')

In [1]:
# Adversarial training of CIFAR-CNN and CIFAR-CNN-With_BatchNorm

In [9]:
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 98.15 82.28%

Training and Test accuracies: 2.99 5.60%
Epoch 1/1
391/391 [==============================] - 24s 63ms/step - loss: 2.3285 - acc: 0.4165
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6557 - acc: 0.9131
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 2.0494 - acc: 0.4871
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6616 - acc: 0.9059
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.9210 - acc: 0.5209
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6272 - acc: 0.9152
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.8297 - acc: 0.5519
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6302 - acc: 0.9160
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.7782 - acc: 0.5667
Epoch 1/1
391/391 [==============

In [10]:
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.44 81.69%

Training and Test accuracies: 2.76 6.11%
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.7139 - acc: 0.5970
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6293 - acc: 0.9154
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.6587 - acc: 0.6102
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6161 - acc: 0.9188
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.5942 - acc: 0.6296
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.6213 - acc: 0.9170
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.5681 - acc: 0.6447
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.5991 - acc: 0.9225
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.5249 - acc: 0.6567
Epoch 1/1
391/391 [==============

In [11]:
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_bn_adv_trained = adv_training_1(cnn_clf_bn, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 95.55 79.74%

Training and Test accuracies: 7.73 9.12%
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.0160 - acc: 0.8220
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.5035 - acc: 0.9438
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 1.0005 - acc: 0.8258
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.4913 - acc: 0.9471
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.9707 - acc: 0.8348
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.4995 - acc: 0.9455
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.9663 - acc: 0.8320
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.4938 - acc: 0.9453
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.9424 - acc: 0.8383
Epoch 1/1
391/391 [==============


Training and Test accuracies: 97.71 79.95%

Training and Test accuracies: 8.91 10.70%
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.6506 - acc: 0.9015
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.3913 - acc: 0.9589
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.6477 - acc: 0.9017
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.3822 - acc: 0.9602
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.6365 - acc: 0.9050
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.3807 - acc: 0.9607
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.6288 - acc: 0.9070
Epoch 1/1
391/391 [==============================] - 23s 59ms/step - loss: 0.3746 - acc: 0.9623
Epoch 1/1
391/391 [==============================] - 23s 58ms/step - loss: 0.6317 - acc: 0.9069
Epoch 1/1
391/391 [==============================

In [15]:
save_clf(cnn_clf_bn, '/home/surthi/models/cifar10/', 'cnn_bn_clf_adv1_trained_final_again1.h5', 'cnn_bn_model_adv1_trained_final_again1.h5')

In [15]:
save_clf(cnn_clf_bn, '/home/surthi/models/cifar10/', 'cnn_bn_clf_adv1_trained_final.h5', 'cnn_bn_model_adv1_trained_final.h5')

In [8]:
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 97.93 81.64%

Training and Test accuracies: 0.42 4.39%
Epoch 1/1
391/391 [==============================] - 16s 40ms/step - loss: 16.1883 - acc: 0.0042
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5026 - acc: 0.9242
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 15.3048 - acc: 0.0614
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 14.5751 - acc: 0.1006
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 14.5523 - acc: 0.0993
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 14.5448 - acc: 0.0987
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 14.4799 - acc: 0.1023
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 14.5039 - acc: 0.1006
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 14.5143 - acc: 0.0998
Epoch 1/1
391/391 [======

In [9]:
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 10.00 10.00%

Training and Test accuracies: 10.00 10.00%
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 14.5257 - acc: 0.0989
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 5.1065 - acc: 0.2584
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 2.3980 - acc: 0.2150
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.5242 - acc: 0.5998
Epoch 1/1
391/391 [==============================] - 15s 37ms/step - loss: 2.3492 - acc: 0.2364
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 1.1915 - acc: 0.7141
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 2.1582 - acc: 0.3097
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.9274 - acc: 0.8125
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 2.0580 - acc: 0.3666
Epoch 1/1
391/391 [===========


Training and Test accuracies: 97.17 81.28%

Training and Test accuracies: 56.67 45.94%
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.3998 - acc: 0.6373
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5578 - acc: 0.9282
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.3872 - acc: 0.6432
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5593 - acc: 0.9319
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.3955 - acc: 0.6425
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5679 - acc: 0.9310
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.3815 - acc: 0.6428
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5410 - acc: 0.9338
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.3700 - acc: 0.6491
Epoch 1/1
391/391 [=============================

In [10]:
save_clf(cnn_clf, '/home/surthi/models/cifar10/', 'cnn_clf_adv1_trained_intermediate.h5', 'cnn_bn_model_adv1_trained_intermediate.h5')

In [11]:
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)


Before training:

Training and Test accuracies: 97.71 80.34%

Training and Test accuracies: 53.46 41.92%
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.2973 - acc: 0.6866
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5574 - acc: 0.9386
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.2783 - acc: 0.6856
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5440 - acc: 0.9402
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.2856 - acc: 0.6913
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5586 - acc: 0.9393
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.2795 - acc: 0.6940
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5375 - acc: 0.9429
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.2596 - acc: 0.6995
Epoch 1/1
391/391 [============


Training and Test accuracies: 98.37 79.72%

Training and Test accuracies: 66.00 44.65%
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1484 - acc: 0.7548
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 0.5592 - acc: 0.9487
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1502 - acc: 0.7549
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5389 - acc: 0.9515
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1369 - acc: 0.7605
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5452 - acc: 0.9518
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1350 - acc: 0.7584
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5623 - acc: 0.9500
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1429 - acc: 0.7603
Epoch 1/1
391/391 [=============================

In [12]:
save_clf(cnn_clf, '/home/surthi/models/cifar10/', 'cnn_clf_adv1_trained_final.h5', 'cnn_bn_model_adv1_trained_final.h5')

In [22]:
cnn_clf, cnn_model = load_clf('/home/surthi/models/cifar10/', 'cnn_clf_adv1_trained_final.h5', 'cnn_bn_model_adv1_trained_final.h5')
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)
cnn_clf_adv_trained = adv_training_1(cnn_clf, g_x_train, g_y_train, g_x_train_bim_adv, g_x_test_bim_adv, g_x_test, g_y_test, epochs=5)

Before training:

Training and Test accuracies: 98.50 79.25%

Training and Test accuracies: 69.39 44.96%
Epoch 1/1
391/391 [==============================] - 16s 41ms/step - loss: 1.0968 - acc: 0.7929
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5806 - acc: 0.9541
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.0920 - acc: 0.7938
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5825 - acc: 0.9545
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.0765 - acc: 0.7986
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5734 - acc: 0.9553
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.0639 - acc: 0.7997
Epoch 1/1
391/391 [==============================] - 15s 39ms/step - loss: 0.5730 - acc: 0.9556
Epoch 1/1
391/391 [==============================] - 15s 38ms/step - loss: 1.1116 - acc: 0.7943
Epoch 1/1
391/391 [============

In [25]:
save_clf(cnn_clf, '/home/surthi/models/cifar10/', 'cnn_clf_adv1_trained_final.h5', 'cnn_bn_model_adv1_trained_final.h5')

# Evaluating Accuracies on CLEAN AND 4 ATTACK ADVERSARIALS

In [31]:
evaluate(cnn_clf_adv_final, g_x_train, g_y_train, g_x_test, g_y_test)
evaluate(cnn_clf_adv_final, g_x_train_bim_adv, g_y_train, g_x_test_bim_adv, g_y_test)
evaluate(resnet_clf_adv_final, g_x_train, g_y_train, g_x_test, g_y_test)
evaluate(resnet_clf_adv_final, g_x_train_bim_adv, g_y_train, g_x_test_bim_adv, g_y_test)


Training and Test accuracies: 98.54 78.86%

Training and Test accuracies: 78.58 46.73%

Training and Test accuracies: 97.27 77.17%

Training and Test accuracies: 71.33 51.09%


(0.71328, 0.5109)

In [ ]:
train_acc, test_acc = evaluate(cnn_clf_adv_final, g_x_train, g_y_train, g_x_test, g_y_test)
fgsm_train_acc, fgsm_test_acc = evaluate(cnn_clf_adv_final, g_x_train_fgsm_adv, g_y_train, g_x_test_fgsm_adv, g_y_test)
ifgsm_train_acc, ifgsm_test_acc = evaluate(cnn_clf_adv_final, g_x_train_ifgsm_adv, g_y_train, g_x_test_ifgsm_adv, g_y_test)
df_train_acc, df_test_acc = evaluate(cnn_clf_adv_final, g_x_train_deepfool_adv, g_y_train[:5000], g_x_test_deepfool_adv, g_y_test[:1000])
cl2_train_acc, cl2_test_acc = evaluate(cnn_clf_adv_final, g_x_train_cl2_adv, g_y_train[:5000], g_x_test_cl2_adv, g_y_test[:1000])

rn_train_acc, rn_test_acc = evaluate(resnet_clf_adv_final, g_x_train, g_y_train, g_x_test, g_y_test)
rn_fgsm_train_acc, rn_fgsm_test_acc = evaluate(resnet_clf_adv_final, g_x_train_fgsm_adv, g_y_train, g_x_test_fgsm_adv, g_y_test)
rn_ifgsm_train_acc, rn_ifgsm_test_acc = evaluate(resnet_clf_adv_final, g_x_train_ifgsm_adv, g_y_train, g_x_test_ifgsm_adv, g_y_test)
rn_df_train_acc, rn_df_test_acc = evaluate(resnet_clf_adv_final, g_x_train_deepfool_adv, g_y_train[:5000], g_x_test_deepfool_adv, g_y_test[:1000])
rn_cl2_train_acc, rn_cl2_test_acc = evaluate(resnet_clf_adv_final, g_x_train_cl2_adv, g_y_train[:5000], g_x_test_cl2_adv, g_y_test[:1000])

In [ ]:
acc1 = (train_acc, df_train_acc, cl2_train_acc, fgsm_train_acc, ifgsm_train_acc)
test_acc1 = (test_acc, df_test_acc, cl2_test_acc, fgsm_test_acc, ifgsm_test_acc)
acc_bn1 = (rn_train_acc, rn_df_train_acc, rn_cl2_train_acc, rn_fgsm_train_acc, rn_ifgsm_train_acc)
test_acc_bn1 = (rn_test_acc, rn_df_test_acc, rn_cl2_test_acc, rn_fgsm_test_acc, rn_ifgsm_test_acc)



In [ ]:


ls_clean_bn = loss_sensitivity(c1_bn, x_train[:5000], y_train[:5000])
ls_clean = loss_sensitivity(c1, x_train[:5000], y_train[:5000])

ls_df_bn = loss_sensitivity(c1_bn, x_train_deepfool_adv_5to10, y_train[5000:10000])
ls_df_bn_2 = loss_sensitivity(c1_bn, x_train_deepfool_adv, y_train[:5000])

ls_cl2_bn = loss_sensitivity(c1_bn, x_train_cl2_adv_5to10, y_train[5000:10000])
ls_cl2_bn_2 = loss_sensitivity(c1_bn, x_train_cl2_adv, y_train[:5000])

ls_fgsm_bn = loss_sensitivity(c1_bn, x_train_fgsm_adv[:5000], y_train[:5000])
ls_ifgsm_bn = loss_sensitivity(c1_bn, x_train_ifgsm_adv[:5000], y_train[:5000])

ls_df = loss_sensitivity(c1, x_train_deepfool_adv_5to10, y_train[5000:10000])
ls_df_2 = loss_sensitivity(c1, x_train_deepfool_adv, y_train[:5000])

ls_cl2 = loss_sensitivity(c1, x_train_cl2_adv_5to10, y_train[5000:10000])
ls_cl2_2 = loss_sensitivity(c1, x_train_cl2_adv, y_train[:5000])

ls_fgsm = loss_sensitivity(c1, x_train_fgsm_adv[:5000], y_train[:5000])
ls_ifgsm = loss_sensitivity(c1, x_train_ifgsm_adv[:5000], y_train[:5000])